In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import scipy.stats as sts
from scipy.stats import binom, multivariate_normal
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# industry = {
#     "2016": [0, 0, 1, 0],
#     "2017": [0, 0, 1, 1, 0],
# }

In [ ]:
probability_grid = np.linspace(0, 1, 1000)
prior = np.ones(1000) / 1000
obs = [0, 1, 0, 0, 0, 0, 1, 1, 0, 1]
total_obs = 0
total_true = 0
idx = 0
current_obs = []
plt.plot(probability_grid, prior)
plt.yticks([])


In [ ]:
if idx < len(obs):
    item = obs[idx]
    if item == 1:
        total_true += 1
    total_obs += 1

    current_obs.append(item)
    title = " ".join([str(i) for i in current_obs])
    likelihood = binom.pmf(total_true, total_obs, p=probability_grid)
    posterior = likelihood * prior
    posterior = posterior / np.sum(posterior)
    plt.plot(probability_grid, prior, linestyle="dashed")
    prior = posterior
    plt.plot(probability_grid, prior)
    plt.title(title)
    print("mean:", np.sum(probability_grid * prior))

    idx = idx+1
else:
    plt.plot(probability_grid, prior)
    plt.title(title)
    print("mean:", np.sum(probability_grid * prior))
plt.yticks([])


In [ ]:
obs = [1, 1, 1, 1, 1, 1]
idx = 0
if idx < len(obs):
    item = obs[idx]
    if item == 1:
        total_true += 1
    total_obs += 1

    current_obs.append(item)
    title = " ".join([str(i) for i in current_obs])
    likelihood = binom.pmf(total_true, total_obs, p=probability_grid)
    posterior = likelihood * prior
    posterior = posterior / np.sum(posterior)
    plt.plot(probability_grid, prior, linestyle="dashed")
    prior = posterior
    plt.plot(probability_grid, prior)
    plt.title(title)
    print("mean:", np.sum(probability_grid * prior))

    idx = idx+1
else:
    plt.plot(probability_grid, prior)
    plt.title(title)
    print("mean:", np.sum(probability_grid * prior))
plt.yticks([])


In [ ]:
likelihood = binom.pmf(total_true, total_obs, p=probability_grid)
prior = np.ones(1000) / 1000
posterior = likelihood * prior
posterior = posterior / np.sum(posterior)
# print(np.var(posterior))
# prior = posterior
plt.plot(probability_grid, posterior)
print("mean:", np.sum(probability_grid * posterior))
plt.yticks([])

In [ ]:
dataset = pd.read_csv('./Salary_Data.csv')
X = dataset.iloc[:, :-1].values
y = dataset.iloc[:, 1].values / 10000

# Fit linear regression

from sklearn.linear_model import LinearRegression
regressor = LinearRegression()
regressor.fit(X, y)

# Predicting the Test set results
y_pred = regressor.predict(X)

In [ ]:
plt.scatter(X, y, c='b', label='Data')
plt.plot(X, y_pred, c='r', label='Linear regression')
plt.xlabel('Years of experience')
plt.ylabel('Salary')
plt.legend()

In [ ]:
# Compute standard errors
noise_std = np.sqrt(np.mean((y - y_pred)**2))
print('Noise std: {:.3f}'.format(noise_std))

X_1 = np.concatenate((X, np.ones_like(X)), axis=-1)
H = np.linalg.inv(X_1.T @ X_1) @ X_1.T
A = (X_1 @ H)** 0.5 * noise_std
se = np.diag(A)
se

In [ ]:
plt.scatter(X, y, c='b', label='data')
plt.plot(X, y_pred, c='r', label='Linear regression')
plt.fill_between(X.reshape(-1), (y_pred - 1.96 * se), (y_pred + 1.96 * se), color='r', alpha=.2)

In [ ]:
class ridge:
    def __init__(self, likelihood_std=None, prior_mean=None, prior_std=None) -> None:
        self.prior_mean = prior_mean
        self.prior_std = prior_std
        self.likelihood_std = likelihood_std
        
    def fit(self, X, y):
        prior_var = self.prior_std**2

        if self.likelihood_std is None:
            likelihood_var = np.mean((y - np.mean(y))**2)
            self.likelihood_std = np.sqrt(likelihood_var)
            print(self.likelihood_std)
        else:
            likelihood_var = self.likelihood_std**2

        X = np.concatenate((X, np.ones((X.shape[0], 1))), axis=1)
        self.S = np.linalg.inv(X.T.dot(X) + np.identity(X.shape[1]) * likelihood_var / prior_var)
        beta = self.S.dot(X.T.dot(y) + self.prior_mean * likelihood_var / prior_var)
        self.beta = beta
        print(self.S, beta)
    
    def predict(self, X):
        return np.concatenate((X, np.ones((X.shape[0], 1))), axis=1).dot(self.beta)
        return X.dot(self.beta)
    
    def sample(self, X, k=1000):
        X = np.concatenate((X, np.ones((X.shape[0], 1))), axis=1)
        # beta = multivariate_normal.rvs(self.prior_mean, self.prior_std**2 * np.eye(len(self.prior_mean)), k)
        # prob = multivariate_normal.pdf(beta, self.prior_mean, self.prior_std**2 * np.eye(len(self.prior_mean)))
        mean = X.dot(self.beta)
        sigma = X.dot(self.S).dot(X.T) +self.likelihood_std**2
        y = multivariate_normal.rvs(mean, sigma, k)

        # return X[:, 0], y, np.sum(prob * y, axis=1)
        return X, y.T, mean[:, np.newaxis]
    
    def draw_posterior(self):
        beta = multivariate_normal.rvs(self.beta, self.S, 1000)
        densitiy = multivariate_normal.pdf(beta, self.beta, self.S)


        beta_prior = multivariate_normal.rvs(self.prior_mean, self.prior_std**2 * np.eye(len(self.prior_mean)), 1000)
        densitiy_prior = multivariate_normal.pdf(beta, self.prior_mean, self.prior_std**2 * np.eye(len(self.prior_mean)))

        plt.scatter(beta[:, 0], densitiy, c='b', label='data')
        plt.scatter(beta_prior[:, 0], densitiy_prior, c='r', label='data')

model = ridge(likelihood_std=1, prior_mean=np.array([0, 0]), prior_std=1)
model.fit(X, y)


In [ ]:
model.draw_posterior()

In [ ]:
k=50
X_sample, y_sample, y_mean = model.sample(X, k=k)
y_std = np.sqrt(((y_sample - y_mean)**2).mean(axis=1))

In [ ]:
plt.plot(X[:, 0], y_mean[:, 0])
# plt.scatter(np.repeat(X_sample[:, np.newaxis, 0], k), y_sample, c='r', s=0.5,  label='Sample')
plt.scatter(X, y, c='b', label='Data')
plt.fill_between(X[:, 0], y_mean[:, 0] - y_std, y_mean[:, 0] + y_std, alpha=0.2)

In [ ]:
plt.plot(X[:, 0], y_mean[:, 0])
plt.scatter(np.repeat(X_sample[:, np.newaxis, 0], k), y_sample, c='r', s=0.5,  label='Sample')
plt.scatter(X, y, c='b', label='Data')
plt.fill_between(X[:, 0], y_mean[:, 0] - y_std, y_mean[:, 0] + y_std, alpha=0.2)